In [29]:
from IPython.core.display import HTML
def css_styling():
    styles = open("/Users/jeremynixon/Dropbox/python/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [19]:
import numpy as np
import pandas as pd
import math
import random

In [40]:
import sklearn.cross_validation
iris = pd.read_csv('iris.data', header=None)
# iris = iris[iris[4] != 'Iris-setosa']
# iris = iris.reset_index(drop=True)
y = iris[4]
iris = iris.drop([4], 1)
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(iris, y, test_size = .20, random_state=42)

In [112]:
votes = pd.read_csv('votes.csv')
votes = votes.replace('?', float('NaN'))
votes = votes.dropna()
votes = votes.reset_index(drop=True)
labels = votes['0']
votes = votes.drop('0', axis = 1)
votes = votes.replace('y', 1)
votes = votes.replace('n', 0)
votes = np.array(votes)

In [173]:
class Tree(object):
    def __init__(self, parent = None):
        self.parent = parent
        self.children = []
        self.label = None
        self.split_feature = None
        self.split_feature_value = None

def data_to_density(y_train):
    density = []
    for type in set(y_train):
        density.append(float(list(y_train).count(type))/len(y_train))
    return density

def entropy(probabilities):
    return -sum(p * math.log(p, 2) for p in probabilities)

def split_data(x_train, y_train, feature_index):
    x_train = np.array(x_train)
    for label in set(x_train[:, feature_index]):
        data_subset = []
        for index, datapoint in enumerate(x_train):
            if datapoint[feature_index] == label:
                data_subset.append([datapoint, index])
        yield data_subset
        
def gain(x_train, y_train, feature_index):
    entropy_gain = entropy(data_to_density(y_train))
    for data_subset in split_data(x_train, y_train, feature_index):
        entropy_gain -= entropy(data_to_density([label for (point, label) in data_subset]))
    return entropy_gain

def homogeneous(y_train):
    return len(set(y_train)) <= 1

def majority_vote(y_train, node):
    choice = max(set(y_train), key=list(y_train).count)
    node.label = choice
    return node

def build_decision_tree(x_train, y_train, root, remaining_features):
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    if homogeneous(y_train):
        root.label = y_train[0]
        return root
    
    if len(remaining_features) == 0:
        return majority_vote(y_train, root)
    
    best_feature = max(remaining_features, key = lambda index: 
                       gain(x_train, y_train, index))
    
    if gain(x_train, y_train, best_feature) == 0:
        return majority_vote(y_train, root)
    
    root.split_feature = best_feature
    
    for data_subset in split_data(x_train, y_train, best_feature):
        child = Tree(parent=root)
        child.split_feature_value = data_subset[0][0][best_feature]
        root.children.append(child)
        
        new_x = np.array([point for (point, label) in data_subset])
        new_y = np.array([label for (point, label) in data_subset])
        
        build_decision_tree(new_x, new_y, child, remaining_features - set([best_feature]))
        
        return root
    
def decision_tree(x_train, y_train):
    return build_decision_tree(x_train, y_train, Tree(), set(range(len(x_train[0]))))

def classify(tree, point):
    if tree.children == []:
        return tree.label
    
    else:
        for child in tree.children:
            print child.split_feature_value
            print point[tree.split_feature]
        matching_children = [child for child in tree.children
                if child.split_feature_value == point[tree.split_feature]]
        return classify(matching_children[0], point)
    
def text_classification(array, value):
    nearest = (np.abs(array-value)).argmin()
    return array[nearest]

In [174]:
tree = decision_tree(x_train, y_train)
for test in x_test:
    print classify(tree, x_test)

2.5
[ 5.7  3.8  1.7  0.3]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [145]:
gain(x_train, y_train, 2)

-48.69017900257295

In [135]:
entropy(data_to_density(labels))

0.9965663713274553